In [ ]:
from rawio.IO import *
import ROOT 
import root_numpy

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
pd.set_option("display.max_row", 100)
import numpy as np

import sys
import os, os.path 
import fnmatch
import glob

In [ ]:
DMC1752 = 'data/FastDMC/Pulses-10keV-elog1752_51230925_0000.root'
DMCprod = 'data/FastDMC/Pulses-10keV-V09-13-00_51230925_0000.root'
UCBC = 'data/HVeV_fits/UCB-C-1.95eV_51231212_000000.root'
# -----------------------------------------------------------------------------------------------------------------------------

# Event Number of Interest
event = 0

In [ ]:
# Will want to get binning in getTraces() to eliminate this hardcoding
###chargeBins = np.arange(-409600.0, 1228800, 800) * 1e-3
###phononBins = np.arange(-819200.0, 5734400, 1600) * 1e-3
phononBins = np.arange(-819200.0, 2867200, 800) * 1e-3     # For UCB-C chip

# Will want to get channel names from G4SettingsInfo/Geometry.ChanName list
###chargeChans = ['Qo1','Qi1','Qo2','Qi2']
###phononChans = ['PAS1','PBS1','PCS1','PDS1','PAS2','PBS2','PCS2','PDS2']
phononChans = ['Ch1']

In [ ]:
# Generic function used by getTES() and getFET() below
# tree="g4dmcTES" or "g4dmcFET"; function will take care of directory
def getTraces(file, tree, chans, dtype=0):
    traces = { chan: list() for chan in chans }      # Separate empty lists for each key

    data = root_numpy.root2array(file, treename="G4SimDir/"+tree, 
                                 branches = ['EventNum','Trace','ChanName'],
                                 selection=f"DataType=={dtype}")
    
    # Copy traces for each event into new array indexed by event number
    for i in range(len(data)):
        for ch in traces.keys():
            if data['ChanName'][i] == bytes(ch, "ascii"):
                traces[ch].append(data['Trace'][i])
    
    for ch in traces.keys():
        traces[ch] = array(traces[ch])

    return traces

In [ ]:
def getTES(file, dtype=0):
    tes = getTraces(file, "g4dmcTES", phononChans, dtype)

    # Convert TES traces from downward to upward going, remove baseline offset
    for ch in tes.keys():
        tes[ch] = array([max(tr)-tr for tr in tes[ch]])
        
    return tes

In [ ]:
def getFET(file, dtype=0):
    fet = getTraces(file, "g4dmcFET", chargeChans, dtype)
    return fet

In [ ]:
###TESprod = getTES(DMCprod)
###FETprod = getFET(DMCprod)

In [ ]:
###TES1752 = getTES(DMC1752)
###FET1752 = getFET(DMC1752)

In [ ]:
TES_UCBC = getTES(UCBC)

In [ ]:
#### Only one channel to be plotted for UCB-C
fig1, axes = subplots(1,1,figsize=(4,3), dpi=500)
axes.plot(phononBins, TESprod[phononChans[0]][event])
axes.set_xlabel("Time [\u03BCs]")
axes.set_ylabel("TESSim [\u03BCA]")
axes.set_title(f"Trace for UCB-C {phononChans[0]} in Event {event}")

In [ ]:
fig1.savefig("UCBC_trace.eps")
fig1.savefig("UCBS_trace.png")

In [ ]:
#### All channels on one giant plot -- NOTE dimensions are Y x X
#### NOTE:  This is only good for an iZIP5 detector
fig1, axes = subplots(3,4,figsize=(18,9), dpi = 1000)
fig1.set_tight_layout(True)

# -----------------------------------------------------------------------------------------------------------------------------

# Do the four charge channels first, across the top
for plot in range(4):
    currentAxis = axes.flatten()[plot]
    currentAxis.plot(chargeBins, FETprod[chargeChans[plot]][event], 
                     lw=1, color = 'C0', label='V09-13-00')
    currentAxis.plot(chargeBins, FET1752[chargeChans[plot]][event], 
                     lw=1, color = 'C1', label='elog/1752')
    if plot==0: currentAxis.legend(loc="lower right")
    currentAxis.set_xlabel("Time [\u03BCs]")
    currentAxis.set_ylabel("FETSim [\u03BCV]")
    currentAxis.set_title('Trace for ' + chargeChans[plot] + ' in Event ' + str(event))

# Do the eight phonon channels, side 1 then side 2
for plot in range(4,12):
    currentAxis = axes.flatten()[plot]
    currentAxis.plot(phononBins, TESprod[phononChans[plot-4]][event], 
                     lw=1, color = 'C0', label='V09-13-00')
    currentAxis.plot(phononBins, TES1752[phononChans[plot-4]][event], 
                     lw=1, color = 'C1', label='elog/1752')

    currentAxis.set_xlabel("Time [\u03BCs]")
    currentAxis.set_ylabel("TESSim [\u03BCA]")
    currentAxis.set_title('Trace for ' + phononChans[plot-4] + ' in Event ' + str(event))